In [1]:
using Pkg, Revise
Pkg.activate(".")
using Clapeyron, GCIdentifier, ChemicalIdentifiers
using Plots

  Activating project at `~/Library/CloudStorage/OneDrive-CaliforniaInstituteofTechnology/University/UROP/SAFT_codes/Clapeyron`


In [25]:
revise(Clapeyron)

true

# Neutral Systems

In [ ]:
species = ["ethanol","acetic acid","ethyl acetate","water"]
components = get_groups_from_name.(species,UNIFAC)
reactions = [("ethanol"=>-1,"acetic acid"=>-1,"ethyl acetate"=>1,"water"=>1)]

eosmodel = UNIFAC(components)
model = ReactiveModel(species,reactions; model = eosmodel)

T = LinRange(273.15,373.15,100)
ξ = zeros(100)
z01 = 4e-1
z02 = 4e-1
n0 = [0.5,0.5,0.0,0.0]
for i in 1:100
    ξ[i] = equilibrate(model,1e5,T[i],n0; z0=[z01])[3]
    z01 = ξ[i]
end

In [ ]:
plt = plot(grid=:off,framestyle=:box,foreground_color_legend = nothing,legend_font=font(12), guidefontsize = 12, tickfontsize = 9)
plot!(plt,T,ξ/0.5,label="UNIFAC",linewidth=2)
# plot!(plt,T,ξ[:,2]/0.5,label="PC-SAFT",linewidth=2)
xlims!(plt,273.15,333.15)
xlabel!(plt,"Temperature (K)")
ylabel!(plt,"Conversion")
title!(plt,"Ethanol + Acetic Acid ↔ Ethyl Acetate + Water")
savefig(plt,"ethanol_acetic_acid_ethyl_acetate_water.pdf")

# Aqueous Systems

In [ ]:
species = ["water","hydronium","hydroxide"]
reactions = [("water"=>-2,"hydronium"=>1,"hydroxide"=>1)]

eosmodel = SAFTVREMie(["water"],["hydronium","hydroxide"])
model = ReactiveAqModel(species,reactions; model = eosmodel)

n0 = [1.0,0.0,0.0]

T = LinRange(273.15,333.15,100)
pH = zeros(100)
z0 = 1e-7
for i in 1:100
    pH[i] = Clapeyron.pH(model,1e5,T[i],n0; z0=[z0])
end

In [ ]:
plt = plot(grid=:off,framestyle=:box,foreground_color_legend = nothing,legend_font=font(12), guidefontsize = 12, tickfontsize = 9)
plot!(plt,T,pH,label="SAFT-VRE Mie",linewidth=2)
# plot!(plt,T,ξ[:,2]/0.5,label="PC-SAFT",linewidth=2)
xlims!(plt,273.15,333.15)
xlabel!(plt,"Temperature (K)")
ylabel!(plt,"pH")
title!(plt,"pH of pure water")
# savefig(plt,"pH.pdf")
plt

In [ ]:
species = ["water","hydronium","hydroxide","sodium"]
reactions = [("water"=>-2,"hydronium"=>1,"hydroxide"=>1)]

eosmodel = SAFTVREMie(["water"],["hydronium","hydroxide","sodium"])
model = ReactiveAqModel(species,reactions; model = eosmodel)

salts = [("sodium hydroxide",["sodium"=>1,"hydroxide"=>1]),
         ("h3o.oh",["hydronium"=>1,"hydroxide"=>1])]

m = 10 .^LinRange(-7,-1,100)
T = 298.15

pH = zeros(100)
i = 1

for i in 1:100
    z0 = molality_to_composition(model.eosmodel, salts, [m[i],0.0])

    pH[i] = Clapeyron.pH(model,1e5,T,z0; z0=[1e-7])
end

In [ ]:
plt = plot(grid=:off,framestyle=:box,foreground_color_legend = nothing,legend_font=font(12), guidefontsize = 12, tickfontsize = 9, xaxis=:log)
plot!(plt,m,pH,label="SAFT-VRE Mie",linewidth=2)
xlabel!(plt,"NaOH molality (mol/kg)")
ylabel!(plt,"pH")

In [ ]:
species = ["water","hydronium","hydroxide","chloride"]
reactions = [("water"=>-2,"hydronium"=>1,"hydroxide"=>1)]

eosmodel = SAFTVREMie(["water"],["hydronium","hydroxide","chloride"])
model = ReactiveAqModel(species,reactions; model = eosmodel)

salts = [("hydrogen chloride",["hydronium"=>1,"chloride"=>1]),
         ("h3o.oh",["hydronium"=>1,"hydroxide"=>1])]

m = 10 .^LinRange(-7,-1,100)
T = 298.15

pH = zeros(100)
i = 1

for i in 1:100
    z0 = molality_to_composition(model.eosmodel, salts, [m[i],0.0])

    pH[i] = Clapeyron.pH(model,1e5,T,z0; z0=[1e-7])
end

In [ ]:
plt = plot(grid=:off,framestyle=:box,foreground_color_legend = nothing,legend_font=font(12), guidefontsize = 12, tickfontsize = 9, xaxis=:log)
plot!(plt,m,pH,label="SAFT-VRE Mie",linewidth=2)
xlabel!(plt,"HCl molality (mol/kg)")
ylabel!(plt,"pH")

In [7]:
species = ["water","acetic acid","hydronium","hydroxide","acetate"]
reactions = [("water"=>-2,"hydronium"=>1,"hydroxide"=>1),
             ("water"=>-1,"acetic acid"=>-1,"acetate"=>1,"hydronium"=>1)]

eosmodel = SAFTgammaEMie(["water","acetic acid"],["hydronium","hydroxide","acetate"])
model = ReactiveAqModel(species,reactions; model = eosmodel)

salts = [("acetic acid",["acetate"=>1,"hydronium"=>1]),
         ("h3o.oh",["hydronium"=>1,"hydroxide"=>1])]

z0 =[0.9,0.1,0.0,0.0,0.0]

# z0 = molality_to_composition(model.eosmodel, salts, [0.0,1e-14,0.0], [0.98,0.02])

Clapeyron.pH(model,1e5,298.15,z0; z0=[1e-7,1e-4])

[1.0110054626220328e-14 1.7519664617156773e-5]
Results of solving non-linear equations

* Algorithm:
  Newton's method with default linsolve with Trust Region (Newton, eigen)

* Candidate solution:
  Final residual 2-norm:      2.61e-13
  Final residual Inf-norm:    2.27e-13

  Initial residual 2-norm:    5.37e+01
  Initial residual Inf-norm:  5.01e+01

* Stopping criteria
  |F(x')|               = 2.27e-13 <= 1.00e-08 (true)

* Work counters
  Seconds run:   4.37e-02
  Iterations:    3



-1.1873918341936291

In [49]:
species = ["water","carbon dioxide","hydronium","hydroxide","bicarbonate","sodium"]
reactions = [("water"=>-2,"hydronium"=>1,"hydroxide"=>1),
             ("water"=>-2,"carbon dioxide"=>-1,"bicarbonate"=>1,"hydronium"=>1)]

eosmodel = SAFTgammaEMie(["water","carbon dioxide"],["hydronium","hydroxide","bicarbonate","sodium"])
model = ReactiveAqModel(species,reactions; model = eosmodel)

salts = [("carbon dioxide",["bicarbonate"=>1,"hydronium"=>1]),
         ("sodium hydroxide",["sodium"=>1,"hydroxide"=>1]),
         ("h3o.oh",["hydronium"=>1,"hydroxide"=>1])]

z0 =[0.999,0.001,0.0,1e-10,0.0,1e-10]
z0 /= sum(z0)

# z0 = molality_to_composition(model.eosmodel, salts, [0.0,1e-5,0.0], [0.99,0.01])

z = Clapeyron.equilibrate(model,1e5,298.15,z0; z0=[1e-7,1e-7])

[1.0110054626220328e-14 4.302121165288985e-7]
Results of solving non-linear equations

* Algorithm:
  Newton's method with default linsolve with Trust Region (Newton, eigen)

* Candidate solution:
  Final residual 2-norm:      4.62e+00
  Final residual Inf-norm:    4.14e+00

  Initial residual 2-norm:    1.11e+01
  Initial residual Inf-norm:  9.98e+00

* Stopping criteria
  |F(x')|               = 4.14e+00 <= 1.00e-08 (false)

* Work counters
  Seconds run:   1.66e+00
  Iterations:    43

[7.589023646643154e-43, 5.826776409944142e-7]


6-element Vector{Float64}:
 0.998998834444918
 0.0009994173221590057
 5.826776409944142e-7
 9.999999998e-11
 5.826776409944142e-7
 9.999999998e-11

In [43]:
Clapeyron.stoichiometric_coefficient(model)

6×2 Matrix{Float64}:
 -2.0  -2.0
  0.0  -1.0
  1.0   1.0
  1.0   0.0
  0.0   1.0
  0.0   0.0

In [ ]:
species = ["water","hydronium","hydroxide"]
reactions = [("water"=>-2,"hydronium"=>1,"hydroxide"=>1)]

eosmodel = SAFTVREMie(["water"],["hydronium","hydroxide"])
model = ReactiveAqModel(species,reactions; model = eosmodel)

Clapeyron.equilibrate(model,1e5,298.15,[1.0,0.0,0.0]; z0=[1e-7])